In [12]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [13]:
 #  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie)
    return movie

In [27]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def extract_transform_load():
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(f'{file_dir}movies_metadata.csv', low_memory=False)
    ratings = pd.read_csv(f'{file_dir}ratings.csv')

    # Open and read the Wikipedia data JSON file.
    with open(f'{file_dir}wikipedia.movies.json', mode='r') as file:
        wiki_movies_raw = json.load(file)
    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    # Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw
               if ('Director' in movie or 'Directed by' in movie)
                   and 'imdb_link' in movie
                   and 'No. of episodes' not in movie]

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_movies = [clean_movie(movie) for movie in wiki_movies]

    # Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(clean_movies)
    sorted(wiki_movies_df.columns.tolist())

    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        print(len(wiki_movies_df))
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        print(len(wiki_movies_df))
        wiki_movies_df.head()  
        
    except ValueError:
        print("Oops! There is an error.  Try again....") 

    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
    wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]

    # Create a variable that will hold the non-null values from the “Box office” column.
    box_office = wiki_movies_df['Box office'].dropna()

    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office = box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\d+\.?\d*\s*[mb]illion'
    box_office.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1,3}(?:,\d{3})+'
    box_office.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()

    # Add the parse_dollars function.
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a million
            value = float(s) * 10**6

            # return value
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

            # convert to float and multiply by a billion
            value = float(s) * 10**9

            # return value
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            # remove dollar sign and commas
            s = re.sub('\$|,','', s)

            # convert to float
            value = float(s)

            # return value
            return value

        # otherwise, return NaN
        else:
            return np.nan
        
    # Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Box office', axis=1, inplace=True)

    # Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    #wiki_movies_df.drop('Budget', axis=1, inplace=True)

    # Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

    # Clean the running time column in the wiki_movies_df DataFrame.
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    #wiki_movies_df.drop('Running time', axis=1, inplace=True)
     
    # 2. Clean the Kaggle metadata.
    kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])

    # 3. Merged the two DataFrames into the movies DataFrame.
    movies_df = pd.merge(wiki_movies_df, kaggle_metadata, on='imdb_id', suffixes=['_wiki','_kaggle'])
    movies_df[['title_wiki','title_kaggle']]
    movies_df[movies_df['title_wiki'] != movies_df['title_kaggle']][['title_wiki','title_kaggle']]
    movies_df[movies_df['release_date_wiki'].isnull()]
    

    
    # 4. Drop unnecessary columns from the merged DataFrame.
    #movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)
    #movies_df['original_language'].value_counts(dropna=False)
    #movies_df[['Production company(s)','production_companies']]
    #movies_df.drop(columns=['title_wiki','release_date_wiki','Language','Production company(s)'], inplace=True)
    
    movies_df['Language'].value_counts()
    movies_df['Language'].apply(lambda x: tuple(x) if type(x) == list else x).value_counts(dropna=False)
    movies_df['original_language'].value_counts(dropna=False)
    movies_df.drop(columns=['title_wiki','release_date_wiki','Language'], inplace=True)

    # 5. Add in the function to fill in the missing Kaggle data.
    def fill_missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(
            lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column]
            , axis=1)
        df.drop(columns=wiki_column, inplace=True)

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    fill_missing_kaggle_data(movies_df, 'runtime', 'running_time')
    fill_missing_kaggle_data(movies_df, 'budget_kaggle', 'budget_wiki')
    fill_missing_kaggle_data(movies_df, 'revenue', 'box_office')
    movies_df
    # 7. Filter the movies DataFrame for specific columns.
    for col in movies_df.columns:
        lists_to_tuples = lambda x: tuple(x) if type(x) == list else x
        value_counts = movies_df[col].apply(lists_to_tuples).value_counts(dropna=False)
        num_values = len(value_counts)
    if num_values == 1:
        print(col)
        
    movies_df['video'].value_counts(dropna=False)

    # 8. Rename the columns in the movies DataFrame.
    movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
                       'runtime','revenue','popularity','vote_average','vote_count',
                       'genres','original_language','overview','spoken_languages',
                       'production_companies','production_countries']]
    
    movies_df.rename({'id':'kaggle_id',
                  'title_kaggle':'title',
                  'url':'wikipedia_url',
                  'budget_kaggle':'budget',
                  'release_date_kaggle':'release_date',
                  'Country':'country',
                  'Distributor':'distributor',
                  'Producer(s)':'producers',
                  'Director':'director',
                  'Starring':'starring',
                  'Cinematography':'cinematography',
                  'Editor(s)':'editors',
                  'Writer(s)':'writers',
                  'Composer(s)':'composers',
                  'Based on':'based_on'
                 }, axis='columns', inplace=True)
    

    
    # 9. Transform and merge the ratings DataFrame.
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')
    rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
                .rename({'userId':'count'}, axis=1) \
                .pivot(index='movieId',columns='rating', values='count')
    
    rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]
    
    movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_index=True, right_index=True, how='left')
    #movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)
    
    movies_with_ratings_df= pd.concat([movies_df, rating_counts], ignore_index=True)
    
    return wiki_movies_df, movies_with_ratings_df, movies_df

In [28]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = 'C://Users/Rica6/Desktop/ETL/Resources/'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [29]:
 # 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

7076
7033


In [30]:
 # 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [31]:
 # 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

url  year  \
0  https://en.wikipedia.org/wiki/The_Adventures_o...  1990   
1  https://en.wikipedia.org/wiki/After_Dark,_My_S...  1990   
2   https://en.wikipedia.org/wiki/Air_America_(film)  1990   
3    https://en.wikipedia.org/wiki/Alice_(1990_film)  1990   
4      https://en.wikipedia.org/wiki/Almost_an_Angel  1990   

                               imdb_link                            title  \
0  https://www.imdb.com/title/tt0098987/  The Adventures of Ford Fairlane   
1  https://www.imdb.com/title/tt0098994/             After Dark, My Sweet   
2  https://www.imdb.com/title/tt0099005/                      Air America   
3  https://www.imdb.com/title/tt0099012/                            Alice   
4  https://www.imdb.com/title/tt0099018/                  Almost an Angel   

          Directed by                  Produced by  \
0        Renny Harlin   [Steve Perry, Joel Silver]   
1         James Foley  [Ric Kidney, Robert Redlin]   
2  Roger Spottiswoode               Daniel Melnick   
3         Woody Allen              Robert Greenhut   
4        John Cornell                 John Cornell   

                                Screenplay by                     Story by  \
0  [David Arnott, James Cappe, Daniel Waters]  [David Arnott, James Cappe]   
1                [James Foley, Robert Redlin]                          NaN   
2                  [John Eskow, Richard Rush]                          NaN   
3                                         NaN                          NaN   
4                                         NaN                          NaN   

                                            Based on  \
0                        [Characters, by Rex Weiner]   
1  [the novel, After Dark, My Sweet, by, Jim Thom...   
2             [Air America, by, Christopher Robbins]   
3                                                NaN   
4                                                NaN   

                                            Starring  ...        Language  \
0  [Andrew Dice Clay, Wayne Newton, Priscilla Pre...  ...         English   
1  [Jason Patric, Rachel Ward, Bruce Dern, George...  ...         English   
2  [Mel Gibson, Robert Downey Jr., Nancy Travis, ...  ...  [English, Lao]   
3  [Alec Baldwin, Blythe Danner, Judy Davis, Mia ...  ...         English   
4        [Paul Hogan, Elias Koteas, Linda Kozlowski]  ...         English   

        Budget        Box office   Written by Productioncompanies     imdb_id  \
0  $20 million     $21.4 million          NaN                  NaN  tt0098987   
1   $6 million      $2.7 million          NaN                  NaN  tt0098994   
2  $35 million       $57,718,089          NaN                  NaN  tt0099005   
3  $12 million        $7,331,647  Woody Allen                  NaN  tt0099012   
4  $25 million  $6,939,946 (USA)   Paul Hogan                  NaN  tt0099018   

   box_office      budget release_date running_time  
0  21400000.0  20000000.0   1990-07-11        102.0  
1   2700000.0   6000000.0   1990-05-17        114.0  
2  57718089.0  35000000.0   1990-08-10        113.0  
3   7331647.0  12000000.0   1990-12-25        106.0  
4   6939946.0  25000000.0   1990-12-19         95.0  

[5 rows x 28 columns]

In [32]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

imdb_id  kaggle_id                            title  \
0  tt0098987     9548.0  The Adventures of Ford Fairlane   
1  tt0098994    25501.0             After Dark, My Sweet   
2  tt0099005    11856.0                      Air America   
3  tt0099012     8217.0                            Alice   
4  tt0099018    25943.0                  Almost an Angel   

                    original_title                                  tagline  \
0  The Adventures of Ford Fairlane      Kojak. Columbo. Dirty Harry. Wimps.   
1             After Dark, My Sweet          All they risked was everything.   
2                      Air America  The few. The proud. The totally insane.   
3                            Alice                                      NaN   
4                  Almost an Angel                 Who does he think he is?   

  belongs_to_collection                                      wikipedia_url  \
0                   NaN  https://en.wikipedia.org/wiki/The_Adventures_o...   
1                   NaN  https://en.wikipedia.org/wiki/After_Dark,_My_S...   
2                   NaN   https://en.wikipedia.org/wiki/Air_America_(film)   
3                   NaN    https://en.wikipedia.org/wiki/Alice_(1990_film)   
4                   NaN      https://en.wikipedia.org/wiki/Almost_an_Angel   

                               imdb_link  runtime     revenue  ...  \
0  https://www.imdb.com/title/tt0098987/    104.0  20423389.0  ...   
1  https://www.imdb.com/title/tt0098994/    114.0   2700000.0  ...   
2  https://www.imdb.com/title/tt0099005/    112.0  33461269.0  ...   
3  https://www.imdb.com/title/tt0099012/    102.0   7331647.0  ...   
4  https://www.imdb.com/title/tt0099018/     95.0   6939946.0  ...   

   rating_0.5  rating_1.0  rating_1.5 rating_2.0 rating_2.5 rating_3.0  \
0         NaN         NaN         NaN        NaN        NaN        NaN   
1         NaN         NaN         NaN        NaN        NaN        NaN   
2         NaN         NaN         NaN        NaN        NaN        NaN   
3         NaN         NaN         NaN        NaN        NaN        NaN   
4         NaN         NaN         NaN        NaN        NaN        NaN   

  rating_3.5 rating_4.0 rating_4.5  rating_5.0  
0        NaN        NaN        NaN         NaN  
1        NaN        NaN        NaN         NaN  
2        NaN        NaN        NaN         NaN  
3        NaN        NaN        NaN         NaN  
4        NaN        NaN        NaN         NaN  

[5 rows x 29 columns]

In [33]:
 # 15. Check the movies_df DataFrame. 
movies_df.head()

imdb_id  kaggle_id                            title  \
0  tt0098987       9548  The Adventures of Ford Fairlane   
1  tt0098994      25501             After Dark, My Sweet   
2  tt0099005      11856                      Air America   
3  tt0099012       8217                            Alice   
4  tt0099018      25943                  Almost an Angel   

                    original_title                                  tagline  \
0  The Adventures of Ford Fairlane      Kojak. Columbo. Dirty Harry. Wimps.   
1             After Dark, My Sweet          All they risked was everything.   
2                      Air America  The few. The proud. The totally insane.   
3                            Alice                                      NaN   
4                  Almost an Angel                 Who does he think he is?   

  belongs_to_collection                                      wikipedia_url  \
0                   NaN  https://en.wikipedia.org/wiki/The_Adventures_o...   
1                   NaN  https://en.wikipedia.org/wiki/After_Dark,_My_S...   
2                   NaN   https://en.wikipedia.org/wiki/Air_America_(film)   
3                   NaN    https://en.wikipedia.org/wiki/Alice_(1990_film)   
4                   NaN      https://en.wikipedia.org/wiki/Almost_an_Angel   

                               imdb_link  runtime     revenue  popularity  \
0  https://www.imdb.com/title/tt0098987/    104.0  20423389.0    3.834949   
1  https://www.imdb.com/title/tt0098994/    114.0   2700000.0    7.349189   
2  https://www.imdb.com/title/tt0099005/    112.0  33461269.0   10.274376   
3  https://www.imdb.com/title/tt0099012/    102.0   7331647.0    7.196816   
4  https://www.imdb.com/title/tt0099018/     95.0   6939946.0    2.862209   

   vote_average  vote_count  \
0           6.2        72.0   
1           6.5        17.0   
2           5.3       146.0   
3           6.3        57.0   
4           5.6        23.0   

                                              genres original_language  \
0  [{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...                en   
1  [{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...                en   
2  [{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...                en   
3  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...                en   
4  [{'id': 14, 'name': 'Fantasy'}, {'id': 35, 'na...                en   

                                            overview  \
0  Ford "Mr. Rock n' Roll Detective" Fairlane is ...   
1  The intriguing relationship between three desp...   
2  Air America was the CIA's private airline oper...   
3  Alice Tate, mother of two, with a marriage of ...   
4  Terry Dean is an electronics wizard and thief....   

                                    spoken_languages  \
0           [{'iso_639_1': 'en', 'name': 'English'}]   
1           [{'iso_639_1': 'en', 'name': 'English'}]   
2  [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
3           [{'iso_639_1': 'en', 'name': 'English'}]   
4           [{'iso_639_1': 'en', 'name': 'English'}]   

                                production_companies  \
0  [{'name': 'Twentieth Century Fox Film Corporat...   
1  [{'name': 'Avenue Pictures Productions', 'id':...   
2  [{'name': 'IndieProd Company Productions', 'id...   
3             [{'name': 'Orion Pictures', 'id': 41}]   
4          [{'name': 'Paramount Pictures', 'id': 4}]   

                                production_countries  
0  [{'iso_3166_1': 'US', 'name': 'United States o...  
1  [{'iso_3166_1': 'US', 'name': 'United States o...  
2  [{'iso_3166_1': 'US', 'name': 'United States o...  
3  [{'iso_3166_1': 'US', 'name': 'United States o...  
4  [{'iso_3166_1': 'US', 'name': 'United States o...